In [ ]:
from __future__ import print_function
from __future__ import absolute_import

import os
import numpy as np
import time
import logging
import argparse
from collections import OrderedDict
import faiss

import torch
import torch.nn as nn

from models import SupResNet, SSLResNet
from utils3 import (
    get_features,
    get_features_adv_prob,
    get_features_adv,
    get_roc_sklearn,
    get_pr_sklearn,
    get_fpr,
    get_outliers,
    get_scores_one_cluster,
)
import data

In [ ]:
#Helper functions

In [ ]:
# local utils for SSD evaluation
def get_scores(ftrain, ftest, food, labelstrain, args):
    if args.clusters == 1:
        return get_scores_one_cluster(ftrain, ftest, food)
    else:
        if args.training_mode == "SupCE":
            print("Using data labels as cluster since model is cross-entropy")
            ypred = labelstrain
        else:
            ypred = get_clusters(ftrain, args.clusters)
        return get_scores_multi_cluster(ftrain, ftest, food, ypred)

In [ ]:
def get_clusters(ftrain, nclusters):
    kmeans = faiss.Kmeans(
        ftrain.shape[1], nclusters, niter=100, verbose=False, gpu=False
    )
    kmeans.train(np.random.permutation(ftrain))
    _, ypred = kmeans.assign(ftrain)
    return ypred


def get_scores_multi_cluster(ftrain, ftest, food, ypred):
    xc = [ftrain[ypred == i] for i in np.unique(ypred)]

    din = [
        np.sum(
            (ftest - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (ftest - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]
    dood = [
        np.sum(
            (food - np.mean(x, axis=0, keepdims=True))
            * (
                np.linalg.pinv(np.cov(x.T, bias=True)).dot(
                    (food - np.mean(x, axis=0, keepdims=True)).T
                )
            ).T,
            axis=-1,
        )
        for x in xc
    ]

    din = np.min(din, axis=0)
    dood = np.min(dood, axis=0)

    return din, dood


def get_eval_results(ftrain, ftest, food, labelstrain, args):
    """
    None.
    """
    print(food)
    print(food.shape)
    # standardize data
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)

    dtest, dood = get_scores(ftrain, ftest, food, labelstrain, args)

    fpr95 = get_fpr(dtest, dood)
    auroc, aupr = get_roc_sklearn(dtest, dood), get_pr_sklearn(dtest, dood)
    return fpr95, auroc, aupr

def get_eval_results_adv(ftrain, ftest, food, labelstrain, args):
    """
    None.
    """
    print(food)
    print(food.shape)
    # standardize data
    ftrain /= np.linalg.norm(ftrain, axis=-1, keepdims=True) + 1e-10
    ftest /= np.linalg.norm(ftest, axis=-1, keepdims=True) + 1e-10
    food /= np.linalg.norm(food, axis=-1, keepdims=True) + 1e-10

    m, s = np.mean(ftrain, axis=0, keepdims=True), np.std(ftrain, axis=0, keepdims=True)

    ftrain = (ftrain - m) / (s + 1e-10)
    ftest = (ftest - m) / (s + 1e-10)
    food = (food - m) / (s + 1e-10)

    dtest, dood = get_scores(ftrain, ftest, food, labelstrain, args)

    fpr95 = get_fpr(dtest, dood)
    yo = get_outliers(dtest, dood, percentile=95)
    auroc, aupr = get_roc_sklearn(dtest, dood), get_pr_sklearn(dtest, dood)
    return fpr95, auroc, aupr, dood, dtest, yo


In [ ]:
parser = argparse.ArgumentParser(description="SSD evaluation")

parser.add_argument("--exp-name", type=str, default="temp_eval_ssd")
parser.add_argument(
"--training-mode", type=str,default="SimCLR", choices=("SimCLR", "SupCon", "SupCE")
)
parser.add_argument("--results-dir", type=str, default="./eval_results")

parser.add_argument("--arch", type=str, default="resnet18")
parser.add_argument("--classes", type=int, default=10)
parser.add_argument("--clusters", type=int, default=1)

parser.add_argument("--dataset", type=str, default="cifar100")
parser.add_argument(
"--data-dir", type=str, default="datasets/"
)
parser.add_argument(
"--data-mode", type=str, choices=("org", "base", "ssl"), default="base"
)
parser.add_argument("--normalize", action="store_true", default=True)
parser.add_argument("--batch-size", type=int, default=256)
parser.add_argument("--size", type=int, default=32)

parser.add_argument("--gpu", type=str, default="0")
parser.add_argument("--ckpt", type=str, help="checkpoint path")
parser.add_argument("--seed", type=int, default=12345)

args = parser.parse_args([])
device = "cuda:0"

In [ ]:
args.ckpt= "./saves/c100r18/1--dataset-cifar100-arch-resnet18-lr-0.5_epochs-500/checkpoint/checkpoint_500.pth.tar"


In [ ]:
if not os.path.isdir(args.results_dir):
        os.mkdir(args.results_dir)

results_file = os.path.join(args.results_dir, args.exp_name + "_ssd.txt")

logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger()
logger.addHandler(logging.FileHandler(results_file, "a"))
logger.info(args)

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)


In [ ]:
# create model
if args.training_mode in ["SimCLR", "SupCon"]:
    model = SSLResNet(arch=args.arch).eval()
elif args.training_mode == "SupCE":
    model = SupResNet(arch=args.arch, num_classes=args.classes).eval()
else:
    raise ValueError("Provide model class")
model.encoder = nn.DataParallel(model.encoder).to(device)

In [ ]:
# load checkpoint
ckpt_dict = torch.load(args.ckpt, map_location="cpu")
if "model" in ckpt_dict.keys():
    ckpt_dict = ckpt_dict["model"]
if "state_dict" in ckpt_dict.keys():
    ckpt_dict = ckpt_dict["state_dict"]
model.load_state_dict(ckpt_dict)

In [ ]:
# dataloaders
train_loader, test_loader, norm_layer = data.__dict__[args.dataset](
    args.data_dir,
    args.batch_size,
    mode=args.data_mode,
    normalize=args.normalize,
    size=args.size,
)

features_train, labels_train = get_features(
    model.encoder, train_loader
)  # using feature befor MLP-head
features_test, _ = get_features(model.encoder, test_loader)
print("In-distribution features shape: ", features_train.shape, features_test.shape)

ds = ["cifar10", "cifar100", "stl"]
# ds.remove(args.dataset)


In [ ]:
#Adversarial NMI, ARI, F, and ACC
device = 'cuda'
import models_clu

gen_input_nc = 3
# load the generator of adversarial examples
# pretrained_generator_path = './models/netG_cc_epoch_120.pth'
pretrained_generator_path = '../../Generator_Models/CIFAR100/netG_SPICE_CIFAR20.pth'
pretrained_G = models_clu.Generator(gen_input_nc, 3).to(device)
pretrained_G.load_state_dict(torch.load(pretrained_generator_path))
pretrained_G.eval()

In [ ]:
# for d in ds:
d = 'cifar100'
_, ood_loader, _ = data.__dict__[d](
    args.data_dir,
    args.batch_size,
    mode="base",
    normalize=args.normalize,
    norm_layer=norm_layer,
    size=args.size,
)
features_ood, _ = get_features(model.encoder, ood_loader)
#     features_ood, _ = get_features_adv(model.encoder, ood_loader, pretrained_G)
print("Out-of-distribution features shape: ", features_ood.shape)

fpr95, auroc, aupr = get_eval_results(
    np.copy(features_train),
    np.copy(features_test),
    np.copy(features_ood),
    np.copy(labels_train),
    args,
)

logger.info(
    f"In-data = {args.dataset}, OOD = {d}, Clusters = {args.clusters}, FPR95 = {fpr95}, AUROC = {auroc}, AUPR = {aupr}"
)

In [ ]:
d = 'cifar100'
_, ood_loader, _ = data.__dict__[d](
    args.data_dir,
    args.batch_size,
    mode="base",
    normalize=args.normalize,
    norm_layer=norm_layer,
    size=args.size,
)
#     features_ood, _ = get_features(model.encoder, ood_loader)
features_ood, _ = get_features_adv(model.encoder, ood_loader, pretrained_G)
print("Out-of-distribution features shape: ", features_ood.shape)

fpr95, auroc, aupr = get_eval_results(
    np.copy(features_train),
    np.copy(features_test),
    np.copy(features_ood),
    np.copy(labels_train),
    args,
)

logger.info(
    f"In-data = {args.dataset}, OOD = {d}, Clusters = {args.clusters}, FPR95 = {fpr95}, AUROC = {auroc}, AUPR = {aupr}"
)

In [ ]:
# plt.plot(ood)

In [ ]:
d = 'cifar100'
_, ood_loader, _ = data.__dict__[d](
    args.data_dir,
    args.batch_size,
    
    mode="base",
    normalize=args.normalize,
    norm_layer=norm_layer,
    size=args.size,
)
#     features_ood, _ = get_features(model.encoder, ood_loader)
features_ood, labels_ood = get_features(model.encoder, ood_loader)
print("Out-of-distribution features shape: ", features_ood.shape)

fpr95, auroc, aupr, dood, dtest, yo = get_eval_results_adv(
    np.copy(features_train),
    np.copy(features_test),
    np.copy(features_ood),
    np.copy(labels_train),
    args,
)

logger.info(
    f"In-data = {args.dataset}, OOD = {d}, Clusters = {args.clusters}, FPR95 = {fpr95}, AUROC = {auroc}, AUPR = {aupr}"
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(10000) ,dood)
plt.plot(range(10000) ,dtest)
plt.show()

In [ ]:
d = 'cifar100'
_, ood_loader, _ = data.__dict__[d](
    args.data_dir,
    args.batch_size,
    
    mode="base",
    normalize=args.normalize,
    norm_layer=norm_layer,
    size=args.size,
)
#     features_ood, _ = get_features(model.encoder, ood_loader)
features_ood, labels_ood ,imgc, ad_index, dl2  = get_features_adv_prob(model.encoder, ood_loader, pretrained_G, clamp=0.17)
print("Out-of-distribution features shape: ", features_ood.shape)

fpr95, auroc, aupr, dood, dtest, yo = get_eval_results_adv(
    np.copy(features_train),
    np.copy(features_test),
    np.copy(features_ood),
    np.copy(labels_train),
    args,
)

logger.info(
    f"In-data = {args.dataset}, OOD = {d}, Clusters = {args.clusters}, FPR95 = {fpr95}, AUROC = {auroc}, AUPR = {aupr}"
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(10000) ,dood, color="tab:red", label="Adversarial Distribution", linewidth=2)
plt.plot(range(10000) ,dtest, color="tab:blue", label="Original Distribution", linewidth=2)

plt.margins(0)
plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='off')
# plt.axis('off')

# plt.xticks([])
# plt.yticks([])
plt.ylabel('Feature Values', fontsize= 15)
plt.xlabel('Sample Id', fontsize= 15)
plt.legend(fontsize= 15)
plt.grid()
plt.tight_layout()
plt.savefig('Anomaly_C100_SPICE.pdf', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
len(ad_index)

In [ ]:
co = 0
for j in ad_index:
    if yo[j] == 1:
        co += 1
print(co)

In [ ]:
#averae of 10 runs
avg_l = []

avg_co = []

In [ ]:
for _ in range(100):
    d = 'cifar100'
    _, ood_loader, _ = data.__dict__[d](
        args.data_dir,
        args.batch_size,

        mode="base",
        normalize=args.normalize,
        norm_layer=norm_layer,
        size=args.size,
    )
    #     features_ood, _ = get_features(model.encoder, ood_loader)
    features_ood, labels_ood ,imgc, ad_index, dl2  = get_features_adv_prob(model.encoder, ood_loader, pretrained_G, clamp=0.1)
#     print("Out-of-distribution features shape: ", features_ood.shape)

    fpr95, auroc, aupr, dood, dtest, yo = get_eval_results_adv(
        np.copy(features_train),
        np.copy(features_test),
        np.copy(features_ood),
        np.copy(labels_train),
        args,
    )
    co = 0
    for j in ad_index:
        if yo[j] == 1:
            co += 1

    avg_l += [len(ad_index)]
    avg_co += [co]

In [ ]:
print(avg_l/100, avg_co/100)

In [ ]:
avg_l

In [ ]:
avg_co

In [ ]:
import statistics as st
print(f'Mean {st.mean(avg_l)}, std: {st.stdev(avg_l)}')
print(f'Mean {st.mean(avg_co)}, std: {st.stdev(avg_co)}')